In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [3]:
# Load and preprocess dataset
df = pd.read_csv('scam_messages.csv')
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'].tolist(), df['spam'].tolist(), test_size=0.2)

In [4]:
# Tokenize the texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [5]:
# Create Dataset objects
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask'], 'labels': train_labels})
test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask'], 'labels': test_labels})

In [6]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [8]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [9]:
# Train the model
trainer.train()

TrainOutput(global_step=1146, training_loss=0.1380560950458986, metrics={'train_runtime': 2515.3555, 'train_samples_per_second': 1.822, 'train_steps_per_second': 0.456, 'total_flos': 1205574855659520.0, 'train_loss': 0.1380560950458986, 'epoch': 1.0})

In [10]:
# Save the model
model.save_pretrained('scam_classifier')
tokenizer.save_pretrained('scam_classifier')

('scam_classifier/tokenizer_config.json',
 'scam_classifier/special_tokens_map.json',
 'scam_classifier/vocab.txt',
 'scam_classifier/added_tokens.json')